# Pro Football Reference Web Scraper

This script will scrape data from Pro Football Reference that is useful for fantasy football analysis. There are two main functions below. `scrape_table()` and `scrape_multiple_tables()`

In [1]:
import bs4
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
passing_adv = 'https://www.pro-football-reference.com/years/2021/passing_advanced.htm'
passing = 'https://www.pro-football-reference.com/years/2021/passing.htm'

scraper script dev below

In [ ]:
def clean_text(text):
  '''
  removes special characters in columns
  '''
  # Remove special characters using regex
  cleaned_text = re.sub(r'[^\w\s]', '', text)
  return cleaned_text.strip()

In [ ]:
def find_all_tables(url):
  '''
  extract table objects from url
  '''
  # Send a GET request to the URL
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the HTML content using BeautifulSoup
      soup = BeautifulSoup(response.content, 'html.parser')

      # Find the table in the HTML
      tables = soup.find_all('table')

      return tables
  else:
      print("Failed to fetch the page.")
      return None

In [ ]:
def scrape_table(table):
  # Use Pandas to read the HTML table into a DataFrame
    if table is not None:
        # Find all rows in the table
        rows = table.find_all('tr')

        # Remove the first row
        #rows = rows[1:]

        # Create a list to hold table data
        table_data = []
        for row in rows:
            row_data = []
            # Get all cells in this row
            cells = row.find_all(['th', 'td'])
            for cell in cells:
                # Clean text and remove special characters
                cleaned_cell = clean_text(cell.get_text(strip=True))
                # Append cleaned cell data to row_data list
                row_data.append(cleaned_cell)
            # Append row data to table_data list
            table_data.append(row_data)

        # Convert the table data into a DataFrame
        df = pd.DataFrame(table_data[1:], columns=table_data[0])
        return df
    else:
        print("No table found on the page.")
        return None

In [ ]:
def scrape_multiple_tables(table):
  # Use Pandas to read the HTML table into a DataFrame
    if table is not None:
        # Find all rows in the table
        rows = table.find_all('tr')

        # Remove the first row
        rows = rows[1:]

        # Create a list to hold table data
        table_data = []
        for row in rows:
            row_data = []
            # Get all cells in this row
            cells = row.find_all(['th', 'td'])
            for cell in cells:
                # Clean text and remove special characters
                cleaned_cell = clean_text(cell.get_text(strip=True))
                # Append cleaned cell data to row_data list
                row_data.append(cleaned_cell)
            # Append row data to table_data list
            table_data.append(row_data)

        # Convert the table data into a DataFrame
        df = pd.DataFrame(table_data[1:], columns=table_data[0])
        return df
    else:
        print("No table found on the page.")
        return None

This section loops through all tables and saves data into Pandas dataframe

In [ ]:
passing_dataframes = {}

In [ ]:
tables = find_all_tables(passing)
for table in tables:
  if 'id' in table.attrs:
        table_id = table['id']
        df = scrape_table(table=table)
        df=df[df.Age.apply(lambda x: x.isnumeric())]
        df=df.drop(columns=['Rk'])
        passing_dataframes['passing_df'] = df

In [ ]:
tables = find_all_tables(passing_adv)
for table in tables:
  if 'id' in table.attrs:
        table_id = table['id']
        df = scrape_multiple_tables(table=table)
        df=df[df.Age.apply(lambda x: x.isnumeric())]
        df=df.drop(columns=['Rk'])
        passing_dataframes[f'passing_{table_id}_df'] = df

In [ ]:
passing_dataframes['passing_advanced_accuracy_df']

,Player,Tm,Age,Pos,G,GS,Cmp,Att,Yds,Bats,ThAwy,Spikes,Drops,Drop,BadTh,Bad,OnTgt,OnTgt
0,Tom Brady,TAM,44,QB,17,17,485,719,5316,8,13,2,32,45,127,180,531,754
1,Justin Herbert,LAC,23,QB,17,17,443,672,5014,17,14,1,38,58,96,146,527,802
2,Matthew Stafford,LAR,33,QB,17,17,404,601,4886,9,3,0,31,52,111,186,458,766
3,Patrick Mahomes,KAN,26,QB,17,17,436,658,4839,7,32,0,31,50,118,188,485,775
4,Derek Carr,LVR,30,QB,17,17,428,626,4804,10,21,3,26,43,94,156,466,774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Cordarrelle Patterson,ATL,30,RB,16,13,0,1,0,0,1,0,0,00,0,00,0,00
111,Brett Rypien,DEN,25,QB,1,0,0,2,0,0,1,0,0,00,1,1000,0,00
112,DAndre Swift,DET,22,RB,13,4,0,1,0,0,0,0,0,00,0,00,0,00
113,Albert Wilson,MIA,29,WR,14,5,0,1,0,0,0,0,1,1000,0,00,1,1000


Merging all passing dataframes for a specific year...code in development

In [ ]:
import pandas as pd

def merge_dataframes(dataframes_dict):
    """
    Merge multiple dataframes into one.

    Parameters:
    - dataframes_dict: A dictionary where keys are dataframe names and values are the dataframes.

    Returns:
    - merged_df: The merged dataframe.
    """
    try:
        # Concatenate dataframes along columns (axis=1)
        merged_df = pd.concat(dataframes_dict.values(), axis=1)

        return merged_df
    except Exception as e:
        print(f"Error merging dataframes: {e}")
        return None


In [ ]:
merged_df = merge_dataframes(passing_dataframes)